# Training and Evaluation of GNNs and LLMs
In this notebook, we train the models on the [MovieLens Dataset](https://movielens.org/) after the Pytorch Geometrics Tutorial on [Link Prediction](https://colab.research.google.com/drive/1xpzn1Nvai1ygd_P5Yambc_oe4VBPK_ZT?usp=sharing#scrollTo=vit8xKCiXAue).

First we import all of our dependencies.

The **GraphRepresentationGenerator** manages and trains a GNN model. Its most important interfaces include
**the constructor**, which defines the GNN architecture and loads the pre-trained GNN model if it is already on the hard disk,
**the training method**, which initializes the training on the GNN model and
**the get_embedding methods**, which represent the inference interface to the GNN model and return the corresponding embeddings in the dimension defined in the constructor for given user movie node pairs.

**The MovieLensLoader** loads and manages the data sets. The most important tasks include **saving and (re)loading and transforming** the data sets.

**PromptEncoderOnlyClassifier** and **VanillaEncoderOnlyClassifier** each manage a **prompt (model) LLM** and a **vanilla (model) LLM**. An EncoderOnlyClassifier (ClassifierBase) provides interfaces for training and testing an LLM model.
PromptEncoder and VanillaEncoder differ from their DataCollectors. DataCollectors change the behavior of the models during training and testing and allow data points to be created at runtime. With the help of these collators, we **create non-existent edges on the fly**.

In [1]:
from dataset_manager.movie_lens_manager import MovieLensManager
from dataset_manager.kg_manager import ROOT
from llm_manager.vanilla.classifier import VanillaClassifier

In [2]:
EPOCHS = 2
MODEL_NAME = "google/bert_uncased_L-2_H-128_A-2"
BATCH_SIZE_LLM = 256

We define in advance which **Knowledge Graph Embedding Dimension (KGE_DIMENSION)** the GNN encoder has. We want to determine from which output dimension the GNN encoder can produce embeddings that lead to a significant increase in performance *without exceeding the context length of the LLMs*. In the original tutorial, the KGE_DIMENSION was $64$.

In [3]:
kg_manager = MovieLensManager()

In [4]:
kg_manager.llm_df

,source_id,target_id,split,labels,prompt_feature_title,prompt_feature_genres
0,133620,5270,train,1,About a Boy (2002),"['Comedy', 'Drama', 'Romance']"
1,140111,44131,train,1,Rogue One: A Star Wars Story (2016),"['Action', 'Adventure', 'Fantasy', 'Sci-Fi']"
2,46404,4195,train,1,"Knight's Tale, A (2001)","['Action', 'Comedy', 'Romance']"
3,104802,23160,train,1,The Theory of Everything (2014),"['Drama', 'Romance']"
4,171388,6260,train,1,"Italian Job, The (2003)","['Action', 'Crime']"
...,...,...,...,...,...,...
30720189,127161,75174,val,0,Test (2013),"['Drama', 'Romance']"
30720190,193714,78246,val,0,We Monsters (2015),"['Drama', 'Thriller']"
30720191,131768,16028,val,0,Legend of the Eight Samurai (Satomi hakken-den...,"['Action', 'Adventure', 'Drama', 'Fantasy']"
30720192,165978,62310,val,0,"Geek, and You Shall Find (2019)",['Documentary']


First we load the MovieLensLoader, which downloads the Movie Lens dataset (https://files.grouplens.org/datasets/movielens/ml-32m.zip) and prepares it to be used on GNN and LLM. We also pass the embedding dimensions that we will assume we are training with. First time takes approx. 30 sec.

Next we initialize the vanilla encoder only classifier. This classifier does only use the NLP part of the prompt (no KGE) for predicting if the given link exists.

In [5]:
VANILLA_ROOT = f"{ROOT}/llm/vanilla"

In [6]:
vanilla_bert_classifier = VanillaClassifier(
    df=kg_manager.llm_df,
    source_df=kg_manager.source_df,
    target_df=kg_manager.target_df,
    root_path=VANILLA_ROOT,
    model_name=MODEL_NAME,
)

Some weights of VanillaBertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of VanillaBertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized because the shapes did not match:
- bert.embeddings.token_type_embeddings.weight: found shape torch.Size([2, 128]) in the checkpoint and torch.Size([4, 128]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


device cuda
4


In [7]:
kg_manager.llm_df

,source_id,target_id,split,labels,prompt_feature_title,prompt_feature_genres
0,133620,5270,train,1,About a Boy (2002),"['Comedy', 'Drama', 'Romance']"
1,140111,44131,train,1,Rogue One: A Star Wars Story (2016),"['Action', 'Adventure', 'Fantasy', 'Sci-Fi']"
2,46404,4195,train,1,"Knight's Tale, A (2001)","['Action', 'Comedy', 'Romance']"
3,104802,23160,train,1,The Theory of Everything (2014),"['Drama', 'Romance']"
4,171388,6260,train,1,"Italian Job, The (2003)","['Action', 'Crime']"
...,...,...,...,...,...,...
30720189,127161,75174,val,0,Test (2013),"['Drama', 'Romance']"
30720190,193714,78246,val,0,We Monsters (2015),"['Drama', 'Thriller']"
30720191,131768,16028,val,0,Legend of the Eight Samurai (Satomi hakken-den...,"['Action', 'Adventure', 'Drama', 'Fantasy']"
30720192,165978,62310,val,0,"Geek, and You Shall Find (2019)",['Documentary']


Next we generate a vanilla llm dataset and tokenize it for training.

In [8]:
dataset_vanilla = kg_manager.generate_vanilla_dataset(
    vanilla_bert_classifier.tokenize_function
)

Map:   0%|          | 0/17920114 [00:00<?, ? examples/s]

Map:   0%|          | 0/6400040 [00:00<?, ? examples/s]

Map:   0%|          | 0/6400040 [00:00<?, ? examples/s]

Saving the dataset (0/127 shards):   0%|          | 0/17920114 [00:00<?, ? examples/s]

Saving the dataset (0/45 shards):   0%|          | 0/6400040 [00:00<?, ? examples/s]

Saving the dataset (0/45 shards):   0%|          | 0/6400040 [00:00<?, ? examples/s]

In [9]:
dataset_vanilla["train"][0]["prompt"]

"133620[SEP]5270[SEP]About a Boy (2002)[SEP]['Comedy', 'Drama', 'Romance']"

In [10]:
dataset_vanilla

DatasetDict({
    train: Dataset({
        features: ['source_id', 'target_id', 'split', 'labels', 'prompt_feature_title', 'prompt_feature_genres', 'prompt', 'input_ids', 'attention_mask', 'token_type_ids'],
        num_rows: 17920114
    })
    val: Dataset({
        features: ['source_id', 'target_id', 'split', 'labels', 'prompt_feature_title', 'prompt_feature_genres', 'prompt', 'input_ids', 'attention_mask', 'token_type_ids'],
        num_rows: 6400040
    })
    test: Dataset({
        features: ['source_id', 'target_id', 'split', 'labels', 'prompt_feature_title', 'prompt_feature_genres', 'prompt', 'input_ids', 'attention_mask', 'token_type_ids'],
        num_rows: 6400040
    })
})

Next we train the model on the produced dataset. This can be skipped, if already trained ones.

In [ ]:
vanilla_bert_classifier.train_model_on_data(
    dataset_vanilla, epochs=EPOCHS, batch_size=BATCH_SIZE_LLM
)

  0%|          | 0/140002 [00:00<?, ?it/s]

c:\Users\MARS\.conda\envs\hauptprojekt\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{'loss': 0.6989, 'grad_norm': 1.507749080657959, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.0}
{'loss': 0.7004, 'grad_norm': 1.3345465660095215, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.0}
{'loss': 0.6946, 'grad_norm': 1.485095739364624, 'learning_rate': 3e-06, 'epoch': 0.0}
{'loss': 0.6912, 'grad_norm': 1.3881187438964844, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.0}
{'loss': 0.6864, 'grad_norm': 1.2921674251556396, 'learning_rate': 5e-06, 'epoch': 0.0}
{'loss': 0.682, 'grad_norm': 1.0784025192260742, 'learning_rate': 6e-06, 'epoch': 0.0}
{'loss': 0.6749, 'grad_norm': 1.0453258752822876, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.0}
{'loss': 0.6704, 'grad_norm': 0.7870964407920837, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.0}
{'loss': 0.6646, 'grad_norm': 0.5954148769378662, 'learning_rate': 9e-06, 'epoch': 0.0}
{'loss': 0.6588, 'grad_norm': 0.8096740245819092, 'learning_rate': 1e-05, 'epoch': 0.0}
{'loss': 0.6512, 'grad_norm': 0.608667969

Next we initialize the prompt encoder only classifier. This classifier uses the vanilla prompt and the KGEs for its link prediction.